In [ ]:
from sumo_interfacer import Environment
from Agent import Agent

In [ ]:
N = 20 #Horizon the number of steps after which we perform an update
batch_size = 5
n_epochs = 10
alpha = 0.0003

env = Environment()

agent = Agent(gamma=0.99, alpha=0.0003, gae_lambda=0.95, policy_clip=0.2, batch_size=batch_size, N=N, n_epochs=n_epochs)
n_games = 300
best_score = float('-inf')
score_history = []

learn_iters = 0
avg_score = 0
n_steps = 0

In [ ]:
for i in range(n_games):
    scenario = scenario_generator() #Need to code this
    done = False
    score = 0
    while not done:
        observation = get_state()
        n_steps+=1
        mu_steering, var_steering, steering, mu_acc, var_acc, acc, value = agent.choose_action(observation)
        
        prob_steering = agent.calc_log_prob(mu_steering, var_steering, steering)
        prob_acc = agent.calc_log_prob(mu_acc, var_acc, acc)
        
        observation_, reward, done = env.step(steering, acc)
        score += reward 
        agent.remember(observation, steering, prob_steering, acc, prob_acc, value, reward, done)
        
        if n_steps%N==0:
            agent.learn()
            learn_iters+=1
        
        observation = observation_
    
    score_history.append(score)
    avg_score = np.mean(score_history[-100:])
    
    if best_score<=avg_score:
        best_score = avg_score
        agent.save_models()
        
    print(f'Episode: {n_games}, score: {avg_score}, time_steps: {n_steps}, learning_steps: {learn_iters}')        
        
    